In [0]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(1)

In [6]:
INPUT_DIM = 784
OUTPUT_DIM = 10

class NN(object):
    
    def __init__(self,hidden_dims=(1024,2048),n_hidden=2,init_method='normal',mode='train',datapath=None,model_path=None):
        self.dims = [INPUT_DIM,] + hidden_dims + [OUTPUT_DIM]
        self.weights = []
        self.biases  = []
        
        initialize_weights(n_hidden, self.dims, init_method)
        
        
    def initialize_weights(self, n_hidden, dims, init_method):
        for (input_, output_) in zip(dims[:-1], dims[1:]):
            self.weights.append(init_method(input_, output_))
            self.biases.append(np.zeros(output_))
            
    def forward(self,input_,labels):
        output_ = input_
        for (W, b) in zip(self.weights, self.biases):
            output_ = np.dot(output_, W) + b
        
        output_ = softmax(output_)
        output_loss = loss(output_, labels)
        
        return output_, output_loss
    
    
    def activation(self,input,activate='relu'):
        if activate == 'relu':
            return np.max(input,0)
        
    def softmax(self,input):
        output_ = np.exp(input)
        return output_ / (np.sum(output_, axis=1))
    
    def loss(self, pred, labels):
        ls = np.log(pred)
        ls = np.sum(ls * labels)
        return ls
    
    def backward(self,cache,labels,...):
        
    def update(self,grads,..):

    def train(self):

    def test(self):
        

3